In [1]:
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

In [2]:
current_directory = os.getcwd()

stock_df = pd.read_csv(current_directory + '/input/processed_data/stocks.csv')
tweet_df = pd.read_csv(current_directory + '/input/processed_data/tweets.csv')

In [3]:
stock_df['date_time'] = pd.to_datetime(stock_df['date_time'])
stock_df['date'] = stock_df['date_time'].dt.date
tweet_df['date_time'] = pd.to_datetime(tweet_df['date_time'])
tweet_df['date'] = tweet_df['date_time'].dt.date

In [4]:
total_engagement_threshold = 0
tweet_df = tweet_df[tweet_df['total_engangement'] > total_engagement_threshold]

In [5]:
aggregate_tweet_df = tweet_df.groupby(['ticker_symbol', 'date']).agg({
    'afinn_score': 'mean',
    'vader_sentiment': lambda x: x.mode().iloc[0],
    'tweet': 'count',
    'total_engangement': 'sum'
}).reset_index()

In [6]:
categorical_columns = ['vader_sentiment']
encoder = OneHotEncoder()
aggregate_tweet_df_encoded = encoder.fit_transform(aggregate_tweet_df[categorical_columns])
feature_names = encoder.get_feature_names_out(input_features=categorical_columns)
aggregate_tweet_df = pd.concat([aggregate_tweet_df.drop(categorical_columns, axis=1), pd.DataFrame(aggregate_tweet_df_encoded.toarray(), columns=feature_names)], axis=1)
aggregate_tweet_df.head()

,ticker_symbol,date,afinn_score,tweet,total_engangement,vader_sentiment_Negative,vader_sentiment_Neutral,vader_sentiment_Positive
0,AAPL,2015-01-01,0.477707,155,1699,0.0,1.0,0.0
1,AAPL,2015-01-02,0.383178,319,2373,0.0,1.0,0.0
2,AAPL,2015-01-03,0.845455,110,280,0.0,0.0,1.0
3,AAPL,2015-01-04,0.451852,134,717,0.0,0.0,1.0
4,AAPL,2015-01-05,0.512953,381,1151,0.0,0.0,1.0


In [7]:
aggregate_tweet_df.rename(columns={
    'afinn_score': 'average_afinn_score',
    'vader_sentiment_Negative': 'vader_sentiment_negative',
    'vader_sentiment_Neutral': 'vader_sentiment_neutral',
    'vader_sentiment_Positive': 'vader_sentiment_positive',
    'tweet': 'tweet_count',
    'total_engangement': 'tweet_total_engagement'
}, inplace=True)

In [8]:
tweet_columns = ['ticker_symbol', 'date', 'average_afinn_score', 'vader_sentiment_negative', 'vader_sentiment_neutral', 'vader_sentiment_positive', 'tweet_count', 'tweet_total_engagement']
aggregate_tweet_df = aggregate_tweet_df[tweet_columns]
aggregate_tweet_df = aggregate_tweet_df.reset_index(drop=True)
aggregate_tweet_df.head()

,ticker_symbol,date,average_afinn_score,vader_sentiment_negative,vader_sentiment_neutral,vader_sentiment_positive,tweet_count,tweet_total_engagement
0,AAPL,2015-01-01,0.477707,0.0,1.0,0.0,155,1699
1,AAPL,2015-01-02,0.383178,0.0,1.0,0.0,319,2373
2,AAPL,2015-01-03,0.845455,0.0,0.0,1.0,110,280
3,AAPL,2015-01-04,0.451852,0.0,0.0,1.0,134,717
4,AAPL,2015-01-05,0.512953,0.0,0.0,1.0,381,1151


In [9]:
stock_df['year'] = stock_df['date_time'].dt.year
stock_df['month'] = stock_df['date_time'].dt.month
stock_df['day'] = stock_df['date_time'].dt.day

In [10]:
stock_df.rename(columns = {
    'volume': 'stock_volume',
}, inplace=True)

In [11]:
# stock_columns = ['ticker_symbol', 'date', 'year', 'month', 'day', 'stock_volume', 'close_value', 'prev_close_value']
stock_columns = ['ticker_symbol', 'date', 'close_value']
aggregate_stock_df = stock_df[stock_columns]

In [12]:
merged_df = pd.merge(aggregate_stock_df, aggregate_tweet_df, on=['ticker_symbol', 'date'], how='inner')

In [13]:
date_range = pd.date_range(start='2015-01-01', end='2019-12-31')

grouped = merged_df.groupby('ticker_symbol')

missing_dates_df = pd.DataFrame(columns=['ticker_symbol', 'missing_date'])

for ticker_symbol, group in grouped:
    existing_dates = group['date']
    
    missing_dates = date_range.difference(existing_dates)
    missing_dates_for_ticker = pd.DataFrame({'ticker_symbol': [ticker_symbol] * len(missing_dates),
                                              'missing_date': missing_dates})
    
    missing_dates_df = missing_dates_df.append(missing_dates_for_ticker, ignore_index=True)
    
missing_dates_df.count()

ticker_symbol    11
missing_date     11
dtype: int64

In [14]:
processed_file_path = current_directory + '/input/processed_data/tweets_and_stock.csv'
merged_df.to_csv(processed_file_path, index=False)